In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages (0.0)


In [2]:
# install joblib for saving
# Restart kernel after installing 
!pip install joblib

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
df = pd.read_csv("../data/ml_compile.csv")

In [5]:
df.columns

Index(['year_month', 'land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent', 'bear_rsf_mean', 'bear_rsf_var', 'num_bears',
       'avg_distance_traveled', 'avg_num_land_dens', 'avg_num_ice_dens',
       'avg_num_active_dens'],
      dtype='object')

# Machine Learning: Climate x Resource Selection Factor (RSF)

In [6]:
df = df [['year_month', 'land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent','avg_distance_traveled' ]]

In [9]:
#Drop null values
df = df.dropna(axis=0, thresh = 10)

In [10]:
df.describe()

,land_avg_temp,land_max_temp,land_min_temp,land_ocean_avg_temp,north_min_temp_anomoly,north_max_temp_anomoly,north_mean_temp_anomoly,global_avg_co2,seaice_extent,avg_distance_traveled
count,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,447.000000,8.100000e+01
mean,9.226651,14.930488,3.615477,15.641613,0.775022,0.659465,0.686273,366.108054,11.559153,6.017928e+08
std,4.134906,4.249913,4.010169,1.229699,0.528649,0.567605,0.540999,18.908570,3.185538,6.840158e+08
min,2.558000,8.071000,-2.853000,13.566000,-0.787000,-1.256000,-1.065000,332.410000,3.565600,1.784230e+05
25%,5.189000,10.696000,-0.308000,14.441000,0.413500,0.253000,0.289000,350.320000,8.842585,1.324565e+08
50%,9.394000,15.174000,3.759000,15.683000,0.766000,0.653000,0.671000,364.310000,12.199871,2.650700e+08
75%,13.185500,19.062000,7.447000,16.801500,1.101500,1.037000,1.029500,382.250000,14.409063,9.822863e+08
max,15.482000,21.320000,9.715000,17.611000,2.554000,2.465000,2.510000,403.960000,16.341938,2.617030e+09


## Preprocessing the Data

### Assign X(features) and y(target) data

In [ ]:
X = df_rsf_null[['land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent']]
y_rsf = df_rsf_null["bear_rsf_mean"].values.reshape(-1, 1)
y_var = df_rsf_null["bear_rsf_var"].values.reshape(-1, 1)
print(X.shape, y_rsf.shape)

### Split the data into testing and training dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_rsf_train, y_rsf_test = train_test_split(X, y_rsf, test_size=0.2, random_state=42)
# X_train, X_test, y_var_train, y_var_test = train_test_split(X, y_var, test_size=0.2, random_state=42)

### MinMaxScalar to fit and transform X features and y target

In [ ]:
#Fit Transform using MinMaxScalar for X features
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
y_rsf_minmax = MinMaxScaler().fit(y_rsf_train)
y_var_minmax = MinMaxScaler().fit(y_var_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)

#Target value #1
y_rsf_train_minmax = y_rsf_minmax.transform(y_rsf_train)
y_rsf_test_minmax = y_rsf_minmax.transform(y_rsf_test)

#Target value #2
y_var_train_minmax = y_var_minmax.transform(y_var_train)
y_var_test_minmax = y_var_minmax.transform(y_var_test)

## Multiple Linear Regression Analysis
### Resource Selection Factor Mean

In [ ]:
# Creat model and fit to scaled training data
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_minmax, y_rsf_train_minmax)

In [ ]:
# Make Prediction using a FIT model and plot ==SEE BELOW FOR ALTERNATIVES==
predictions = model.predict(X_test_minmax)
model.fit(X_train_minmax, y_rsf_train_minmax)

plt.scatter(model.predict(X_train_minmax), model.predict(X_train_minmax) - y_rsf_train_minmax, c="blue", label="Training Data")
plt.scatter(model.predict(X_test_minmax), model.predict(X_test_minmax) - y_rsf_test_minmax, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_rsf_test_minmax.min(), xmax=y_rsf_test_minmax.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
# Validate the Model with MSE and R2
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_rsf_test_minmax, predictions)
r2 = model.score(X_test_minmax, y_rsf_test_minmax)

print(f"MSE: {MSE}, R2: {r2}")

### RSF variability

In [ ]:
# Creat model and fit to scaled training data
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_minmax, y_var_train_minmax)

In [ ]:
predictions = model.predict(X_test_minmax)
model.fit(X_train_minmax, y_var_train_minmax)

plt.scatter(model.predict(X_train_minmax), model.predict(X_train_minmax) - y_var_train_minmax, c="blue", label="Training Data")
plt.scatter(model.predict(X_test_minmax), model.predict(X_test_minmax) - y_var_test_minmax, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_var_test_minmax.min(), xmax=y_var_test_minmax.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
# Validate the Model with MSE and R2
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_var_test_minmax, predictions)
r2 = model.score(X_test_minmax, y_var_test_minmax)

print(f"MSE: {MSE}, R2: {r2}")

## Categorical Algorithms
### Preprocessing - Transform RSF values to Categorical

In [ ]:
df["bear_rsf_mean"].fillna(0, inplace=True)

In [ ]:
conditions = [(df["bear_rsf_mean"] < 6), (df["bear_rsf_mean"] > 6) & (df["bear_rsf_mean"] < 10), (df["bear_rsf_mean"] > 10) & (df["bear_rsf_mean"] < 15), (df["bear_rsf_mean"]>15)]
values = ['unknown','low', 'mid', 'high']
df["bear_mean_cat"] = np.select(conditions,values)

In [ ]:
df.head()

In [ ]:
df["bear_mean_cat"].unique()

In [ ]:
#Change Threshold to 9
df_cat_null = df [['land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent','bear_mean_cat']]

In [ ]:
df_cat_null = df_cat_null.dropna(axis=0, how="any")

In [ ]:
df_cat_null.describe()

### Assign new variables

In [ ]:
X = df_cat_null[['land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent']]
y_cat = df_cat_null["bear_mean_cat"]
print(X.shape, y_cat.shape)

### Split the data into testing and training dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_cat_train, y_cat_test = train_test_split(X, y_cat, test_size=0.5, random_state=42)

### MinMaxScalar to fit and transform X features and y target

In [ ]:
#Fit Transform using MinMaxScalar for X features
X_input = []
def rf_algo():
    X_input = []
    X_input_minmax = MinMaxScaler().fit(X_input)
    y_taget = rf.predict(X_input_minmax)
    rf
    
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)

## 1. Logistic Regression Analysis

In [ ]:
# Creat model and fit to scaled training data
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train_minmax, y_cat_train)
print(f"Training Data Score: {classifier.score(X_train, y_cat_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_cat_test)}")

## 2. Random Forest Analysis

In [ ]:
from sklearn.ensemble import RandomForestClassifier
y_cat_rf = df_cat_null["bear_mean_cat"]

rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_minmax, y_cat_train)
score = rf.score(X_test_minmax, y_cat_test)


In [ ]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, X.columns), reverse=True)

In [ ]:
print(f"Random Forest Testing Score: {score}")

### hypertune the model

In [ ]:
rf.get_params().keys()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 3)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

### Label encoding y_cat

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_cat_train)
encoded_y_train = label_encoder.transform(y_cat_train)
encoded_y_test = label_encoder.transform(y_cat_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(X_train_minmax, encoded_y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf2 = RandomForestClassifier(n_estimators=200, min_samples_split =5, min_samples_leaf = 4, max_features='auto', max_depth=60, bootstrap='True')
rf2 = rf2.fit(X_train_minmax, encoded_y_train)
score = rf2.score(X_test_minmax, encoded_y_test)

print(f"Random Forest Testing Score: {score}")

In [ ]:
filename = 'rf_rsf.sav'
joblib.dump("rsf_rsf", filename)

# Machine Learning Prediction: Den location

In [ ]:
df_loc = df[['year_month', 'land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent', 'num_bears',
       'avg_distance_traveled', 'avg_num_land_dens', 'avg_num_ice_dens',
       'avg_num_active_dens']]

In [ ]:
df_loc.describe()

In [ ]:
df_loc_null = df_loc.dropna(axis=0)

In [ ]:
df_loc_null.describe()

## Climate X Bear Mobility 

In [ ]:
X_mob = df_loc_null[['year_month', 'land_avg_temp', 'land_max_temp', 'land_min_temp',
       'land_ocean_avg_temp', 'north_min_temp_anomoly',
       'north_max_temp_anomoly', 'north_mean_temp_anomoly', 'global_avg_co2',
       'seaice_extent']]
y_mob = df_loc_null["avg_distance_traveled"].values.reshape(-1, 1)
print(X_mob.shape, y_mob.shape)

## Land Dens Model

In [ ]:
from sklearn.model_selection import train_test_split
X_mob_train, X_mob_test, y_mob_train, y_mob_test = train_test_split(X_mob, y_mob, test_size=0.2, random_state=42)
# X_train, X_test, y_var_train, y_var_test = train_test_split(X, y_var, test_size=0.2, random_state=42)

In [ ]:
#Fit Transform using MinMaxScalar for X features
from sklearn.preprocessing import MinMaxScaler
X_mob_minmax = MinMaxScaler().fit(X_mob_train)
X_mob_train_minmax = X_mob_minmax.transform(X_mob_train)
X_mob_test_minmax = X_mob_minmax.transform(X_mob_test)

#Target value
y_mob_minmax = MinMaxScaler().fit(y_mob_train)
y_mob_land_train_minmax = y_mob_minmax.transform(y_mob_train)
y_mob_land_test_minmax = y_mob_minmax.transform(y_mob_test)


# Multiple Linear Regression Analysis

## Number of Land Dens

In [ ]:
# Creat model and fit to scaled training data
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_den_train_minmax, y_den_land_train_minmax)

In [ ]:
# Make Prediction using a FIT model and plot
predictions = model.predict(X_den_test_minmax)
model.fit(X_den_train_minmax, y_den_land_train_minmax)

plt.scatter(model.predict(X_den_train_minmax), model.predict(X_den_train_minmax) - y_den_land_train_minmax, c="blue", label="Training Data")
plt.scatter(model.predict(X_den_test_minmax), model.predict(X_den_test_minmax) - y_den_land_test_minmax, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_den_land_test_minmax.min(), xmax=y_den_land_test_minmax.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
# Validate the Model with MSE and R2
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_den_land_test_minmax, predictions)
r2 = model.score(X_den_test_minmax, y_den_land_test_minmax)

print(f"MSE: {MSE}, R2: {r2}")

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

train_scores = []
test_scores = []
for k in range (1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_den_train_minmax, y_den_land_train)
    train_score = knn.score(X_den_train_minmax, y_den_land_train)
    test_score = knn.score(X_den_test_minmax, y_den_land_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
#STEP 5: re-fit classifier with optimal k value
knn = KNeighborsClassifier(n_neighbors = 13)
knn.fit(X_den_train_minmax, y_den_land_train)
print("k=15 Test Acc: %.3f" %knn.score(X_den_test_minmax, y_den_land_test))